In [1]:
import requests
import json
import os
from os.path import join
from typing import Tuple, List, Dict, Union, Set
from time import sleep

In [2]:
api_url = 'http://45.14.48.80:10000/'
upload_url = api_url + 'upload'
task_url = api_url + 'task'
healthcheck_url = api_url + 'healthcheck'
results_url = api_url + 'results'

CURL_POST = 'curl -vvvv -X POST'
CURL_GET = 'curl -vvvv -X GET'

cwd = os.getcwd()
models_path = 'data/models'

In [3]:
models_lst = [el for el in os.listdir(models_path) if el.endswith('.stl')]
models_lst[:2]

['acf7bf74-d74f-4bfa-9141-720ae3dbc73c.stl',
 'db68cd74-4697-409d-bb73-981ba4346c6a.stl']

In [4]:
def upload(full_path: str, upload_url: str) -> Tuple[str, str]:
    r = json.loads(os.popen(f'{CURL_POST} -H "User-Id: test" --form file="@{full_path}" "{upload_url}"').read())
    uuid = r['uuid']
    filename = r['filename']
    return uuid, filename

In [5]:
def task(uuid: str, filename: str, task_url: str):
    config = {
        'uuid' : uuid,
        'verison': '1',
        'file': filename,
        'id': 'test',
        'is_part': False
    }
    config = json.dumps(config)
    os.popen(f'{CURL_POST} -d \'{config}\' "{task_url}"').read()

In [6]:
def healthcheck(healthcheck_url: str) -> Tuple[str, str]:
    r = json.loads(os.popen(f'{CURL_GET} "{healthcheck_url}"').read())
    return r['status'], r['progress']

In [7]:
def get_bins_number_set(results_url: str) -> Set[int]:
    r = json.loads(os.popen(f'{CURL_GET} "{results_url}"').read())['histogram_data']
    return set([el['intervals'] for el in r if not el['type'].startswith('hull')])

In [8]:
def get_results(bins_num: int, results_url: str) -> List[Dict[str, Union[str, List[float]]]]:
    r = json.loads(os.popen(f'{CURL_GET} "{results_url}"').read())['histogram_data']
    return [el for el in r if (el['intervals'] == bins_num) and not el['type'].startswith('hull')]

In [9]:
i = 0
full_path = f'{cwd}/{models_path}/{models_lst[i]}'
uuid, filename = upload(full_path, upload_url)
task(uuid, filename, task_url)
status, progress = healthcheck(healthcheck_url)
while status != 'COMPLETED':
    status, progress = healthcheck(healthcheck_url)
    print(f'progress: {progress}')
    sleep(10)

progress: 13.644926485546577
progress: 20.260648417932796
progress: 27.289852971093154
progress: 34.31905752425351
progress: 41.34826207741387
progress: 48.377466630574226
progress: 54.54830882851552
progress: 61.577513381675885
progress: 68.19323531406211
progress: 75.22243986722246
progress: 82.25164442038282
progress: 89.28084897354319
progress: 96.31005352670354
progress: 100


In [10]:
get_bins_number_set(results_url)

{8, 16, 32, 64, 128}

In [11]:
get_results(8, results_url)

[{'type': 'model_bounding_sphere_strict_outer',
  'intervals': 8,
  'data': [0.025055699471029303,
   0.17532994442600963,
   0.25726323890681246,
   0.37074515993207313,
   0.13983843667827595,
   0.028874494243226134,
   0.0028804782765091837,
   1.2548066064173598e-05]},
 {'type': 'model_bounding_sphere_strict_outer_absolute',
  'intervals': 8,
  'data': [0.004953797451954698,
   0.03466472899522069,
   0.05086387545682938,
   0.07330054508035622,
   0.02764765326560304,
   0.005708816717488275,
   0.000569503396346247,
   2.480895724644832e-06]},
 {'type': 'model_bounding_sphere_missed',
  'intervals': 8,
  'data': [0,
   0,
   0.06518100107198813,
   0.1495148566559468,
   0.1891736071026085,
   0.21859143508974466,
   0.22274195321733872,
   0.15479714686237322]},
 {'type': 'model_bounding_sphere_concentric_sphere',
  'intervals': 8,
  'data': [0.017836378795441428,
   0.1218863368291316,
   0.1523140028051899,
   0.19100526739928783,
   0.16607783704802564,
   0.2210481260160448